In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from glob import glob
from tqdm import tqdm
from torchvision.models import resnet34, ResNet34_Weights

# 환경 설정
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 레이블 추출
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# 파일 리스트 정리
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)
train_files, val_files, train_labels, val_labels = train_test_split(files, labels, test_size=0.2, random_state=42)



In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

# 설정
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# 레이블 추출
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# 파일 분할
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(
    files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42)

# Dataset 클래스
class CTBinaryClassificationDataset(Dataset):
    def __init__(self, file_list, label_list):
        self.files = file_list
        self.labels = label_list

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = np.load(self.files[idx])
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0).astype(np.float32)
        label = self.labels[idx]
        return torch.tensor(img), torch.tensor(label).long()

# CBAM + ResNet18 정의 (생략하지 않고 유지)
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_mlp = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.shared_mlp(self.avg_pool(x))
        max_out = self.shared_mlp(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

class BasicBlockWithCBAM(nn.Module):
    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.cbam = CBAM(planes)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.cbam(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockWithCBAM(self.in_planes, planes, stride, downsample)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockWithCBAM(self.in_planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# 데이터로더
train_loader = DataLoader(CTBinaryClassificationDataset(train_files, train_labels), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(CTBinaryClassificationDataset(val_files, val_labels), batch_size=BATCH_SIZE)
test_loader = DataLoader(CTBinaryClassificationDataset(test_files, test_labels), batch_size=BATCH_SIZE)

# 모델 및 학습 설정
model = ResNet18_CBAM(num_classes=2).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

save_dir = os.path.join(os.path.dirname(os.getcwd()), "pth")
os.makedirs(save_dir, exist_ok=True)
best_val_acc = 0.0

# 학습
for epoch in range(NUM_EPOCHS):
    model.train()
    correct = total = epoch_loss = 0
    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    train_acc = correct / total * 100
    print(f"Train Loss: {epoch_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    # 검증
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total * 100
    print(f"Validation Accuracy: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model_resnet18_CBAM.pth"))
        print("✅ Best model saved!")


# 최종 테스트
print("\n📊 Test Set Evaluation (Best Model 기준):")
model.load_state_dict(torch.load(os.path.join(save_dir, "best_model_resnet18_CBAM.pth")))
model.eval()
y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)[:, 1]  # malignant 확률
        preds = torch.argmax(outputs, dim=1)

        y_probs.extend(probs.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Accuracy 출력
acc = (np.array(y_pred) == np.array(y_true)).mean() * 100
print(f"✅ Test Accuracy: {acc:.2f}%")

# Classification Report
print(classification_report(y_true, y_pred, digits=4))

# AUC 출력
try:
    
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC 계산 실패: 양/음 클래스가 모두 있어야 함.")

# Confusion Matrix 출력
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)


cuda


[Epoch 1]: 100%|██████████| 234/234 [00:13<00:00, 17.86it/s]


Train Loss: 0.6009, Train Acc: 68.06%
Validation Accuracy: 72.12%
✅ Best model saved!


[Epoch 2]: 100%|██████████| 234/234 [00:13<00:00, 17.73it/s]


Train Loss: 0.4671, Train Acc: 78.14%
Validation Accuracy: 72.50%
✅ Best model saved!


...

[Epoch 99]: 100%|██████████| 234/234 [00:12<00:00, 19.17it/s]


Train Loss: 0.0062, Train Acc: 99.79%
Validation Accuracy: 87.88%


[Epoch 100]: 100%|██████████| 234/234 [00:13<00:00, 17.77it/s]


Train Loss: 0.0202, Train Acc: 99.25%
Validation Accuracy: 87.12%

📊 Test Set Evaluation (Best Model 기준):
✅ Test Accuracy: 88.38%
              precision    recall  f1-score   support

           0     0.8640    0.7855    0.8229       275
           1     0.8927    0.9352    0.9135       525

    accuracy                         0.8838       800
   macro avg     0.8784    0.8603    0.8682       800
weighted avg     0.8829    0.8838    0.8823       800

AUC: 0.9303
Confusion Matrix:
[[216  59]
 [ 34 491]]


: 